In [64]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import joblib

In [65]:
# CARREGAMENTO DOS DADOS
train = pd.read_csv('../data/dados_trem_visuais_train.csv')
test = pd.read_csv('../data/dados_trem_visuais_test.csv')

# SEPARANDO FEATURES E TARGET
X_train = train.drop(columns=['precisou_assistencia'])  # substitua "target" pelo nome real da coluna alvo
y_train = train['precisou_assistencia']

X_test = test.drop(columns=['precisou_assistencia'])   # mesmo aqui
y_test = test['precisou_assistencia']


In [66]:
# Identificar colunas categóricas e numéricas
categorical_cols = ["sexo", "linha_metro"]
numerical_cols = [col for col in X_train.columns if col not in categorical_cols]

In [67]:
colunas_numericas = ['idade', 'tempo_de_espera']
colunas_categoricas = [
    'sexo', 'linha_metro', 'estacao_movimentada', 'usa_bengala',
    'aplicativo_acessibilidade', 'horario_pico', 'usa_cao_guia'
]

In [68]:
# Pré-processamento
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), colunas_numericas),
    
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), colunas_categoricas)
])

In [69]:
# Dicionário de modelos para comparação
modelos = {
    "RandomForest": RandomForestClassifier(random_state=42, class_weight="balanced"),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

In [70]:
melhor_modelo = None
melhor_acuracia = 0
melhor_nome = ""

# Loop de avaliação
for nome, modelo in modelos.items():
    pipeline = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", modelo)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    acuracia = accuracy_score(y_test, y_pred)
    
    print(f"\nModelo: {nome}")
    print(f"Acurácia: {acuracia:.4f}")
    print(classification_report(y_test, y_pred))

    if acuracia > melhor_acuracia:
        melhor_acuracia = acuracia
        melhor_modelo = pipeline
        melhor_nome = nome



Modelo: RandomForest
Acurácia: 0.9900
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        58
           1       1.00      0.98      0.99        42

    accuracy                           0.99       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100


Modelo: LogisticRegression
Acurácia: 0.9900
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        58
           1       1.00      0.98      0.99        42

    accuracy                           0.99       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100


Modelo: KNN
Acurácia: 0.9400
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        58
           1       1.00      0.86      0.92        42

    accuracy                           0.94       100
   macro avg  

In [71]:
# Salvando finalmente o modelo
print(f"\n Melhor modelo: {melhor_nome} com acurácia de {melhor_acuracia:.4f}")
joblib.dump(melhor_modelo, "../model/assistencia_model.pkl")


 Melhor modelo: RandomForest com acurácia de 0.9900


['../model/assistencia_model.pkl']